In [2]:
import json
import pprint

with open('cities_usa_Scale8.topo.json') as f:
    d = json.load(f)
    print(d['objects']['citiesScale8']['geometries'][0]["properties"].keys())
#    pprint.pprint(d)

dict_keys(['SCALERANK', 'NATSCALE', 'LABELRANK', 'FEATURECLA', 'NAME', 'NAMEPAR', 'NAMEALT', 'DIFFASCII', 'NAMEASCII', 'ADM0CAP', 'CAPIN', 'WORLDCITY', 'MEGACITY', 'SOV0NAME', 'SOV_A3', 'ADM0NAME', 'ADM0_A3', 'ADM1NAME', 'ISO_A2', 'NOTE', 'LATITUDE', 'LONGITUDE', 'CHANGED', 'NAMEDIFF', 'DIFFNOTE', 'POP_MAX', 'POP_MIN', 'POP_OTHER', 'RANK_MAX', 'RANK_MIN', 'GEONAMEID', 'MEGANAME', 'LS_NAME', 'LS_MATCH', 'CHECKME', 'MAX_POP10', 'MAX_POP20', 'MAX_POP50', 'MAX_POP300', 'MAX_POP310', 'MAX_NATSCA', 'MIN_AREAKM', 'MAX_AREAKM', 'MIN_AREAMI', 'MAX_AREAMI', 'MIN_PERKM', 'MAX_PERKM', 'MIN_PERMI', 'MAX_PERMI', 'MIN_BBXMIN', 'MAX_BBXMIN', 'MIN_BBXMAX', 'MAX_BBXMAX', 'MIN_BBYMIN', 'MAX_BBYMIN', 'MIN_BBYMAX', 'MAX_BBYMAX', 'MEAN_BBXC', 'MEAN_BBYC', 'COMPARE', 'GN_ASCII', 'FEATURE_CL', 'FEATURE_CO', 'ADMIN1_COD', 'GN_POP', 'ELEVATION', 'GTOPO30', 'TIMEZONE', 'GEONAMESNO', 'UN_FID', 'UN_ADM0', 'UN_LAT', 'UN_LONG', 'POP1950', 'POP1955', 'POP1960', 'POP1965', 'POP1970', 'POP1975', 'POP1980', 'POP1985', '

In [4]:
# Manually change some states when misclassified
with open('US_States20190708.json') as f:
    d = json.load(f)
    #print(d)
    for dic in d:
        # substract 252 misclassifications for the city Norman, OK
        if dic["geo_adm1_code"] == "OK": dic["count"] -= 252
        # substract 76 misclassifications for the city Lincoln, NE
        if dic["geo_adm1_code"] == "NE": dic["count"] -= 76
        # substract 540 misclassifications for the city Provo, UT
        if dic["geo_adm1_code"] == "UT": dic["count"] -= 540
        # substract 186 misclassifications for the city Idaho, ID
        if dic["geo_adm1_code"] == "ID": dic["count"] -= 183
        # substract 163 misclassifications for the city Big Spring, TX
        if dic["geo_adm1_code"] == "OK": dic["count"] -= 163         
            
        #print(dic["geo_adm1_code"], dic["count"]/dic["population"])
        print(dic["geo_adm1_code"], dic["count"])
    #print(d)
    
    with open('US_States20190708_manuallyCorrectedSomeMisclassifications.json', 'w') as fi:
        json.dump(d, fi)

MS 545
NE 1247
IA 1795
MT 496
CO 3774
ND 336
WI 2808
FL 7433
NH 386
MN 3817
WY 131
HI 859
CA 18551
VA 3935
ID 704
UT 1501
CT 1442
AR 711
WV 625
NY 15654
ME 614
LA 1831
TN 3175
KY 2100
AL 1051
AK 532
TX 14237
OK 2900
NC 4443
AZ 3560
NM 751
VT 197
DC 4336
SC 944
IN 5320
GA 4978
RI 340
PA 4362
NJ 3130
KS 1376
NV 2584
SD 252
WA 5504
MO 2346
MD 3950
MI 3190
OR 2355
OH 6149
DE 432
MA 5226
IL 7614


In [6]:
with open('US_cities_States20190708.json', "r") as f:
    cities = json.load(f)
    
    with open('cities_usa_Scale8.topo.json') as g:
        topo = json.load(g)
        print("Number cities:", len(cities))
           
        for i in range(len(topo['objects']['citiesScale8']['geometries'])):
            if "North" in topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"]:
                print(topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"])
            # correct wrong geolocalised North Platte to Lincoln in Nebraska
            # And dont consider tweets of Lincoln because most misclassified
            if topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Lincoln" and topo['objects']['citiesScale8']['geometries'][i]["properties"]["ADM1NAME"] == "Nebraska":
                #print(topo['objects']['citiesScale8']['geometries'][i]["properties"])
                    
                for city in cities:
                    if city["city_name"] == "North Platte": 
                        print( city["count"],  city["population"])
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["count"] = city["count"]
                    if city["city_name"] == "Lincoln":
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["population"] = city["population"]
    
            
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Big Spring": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Twin Falls": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "North Platte": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Norman": pass # partially wrong; Oklahoma
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Provo": pass # partially wrong
       
            
            else:
                for city in cities:    
                
                    # Check that New York is considered. Might be written: New York City
                    if city["city_name"] == topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"]:

                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["count"] = city["count"]
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["population"] = city["population"]
            
        print("Write ..")    
        with open('cities_usa_Scale8_CountPopul_20190708.topo.json', 'w', encoding='utf-8') as wr:
            json.dump(topo, wr, ensure_ascii=False, indent=4)

Number cities: 5805
North Platte
193 24194
Write ..
